# Sustainability Anlysis

# 1 Configuration

In [15]:
import os
import pandas as pd
import PyPDF2
import string
import unicodedata
from tqdm import tqdm
from scipy.stats import spearmanr
from scipy.stats import kruskal
import statsmodels.api as sm

# 2 Convert Sustainability Reports to Text Files

In [38]:
def report_to_text(report):
    """Create a text string of pdf-file content"""
    pdfFileObj = open(f"Reports/{report}.pdf", "rb")
    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    no_pages = len(pdfReader.pages)

    report_text = ""
    for i in range(0,no_pages):
        pageObj = pdfReader.pages[i]
        report_text += pageObj.extract_text()
    
    return report_text

In [39]:
file_names = os.listdir("Reports")
report_file_names = [f[:-4] for f in file_names if f[-4:] == ".pdf"]

for report in tqdm(report_file_names):
    report_text = report_to_text(report)
    print(report)
    f = open(f"Reports_text/{report}.txt", "a")
    f.write(report_text)
    f.close()

  0%|          | 0/159 [00:00<?, ?it/s]

TRADERA 2021


FileNotFoundError: [Errno 2] No such file or directory: 'Reports_text/TRADERA 2021.txt'

# 3 Create Sustainability Report Table 

In [41]:
# Create an empty table
report_table = pd.DataFrame(columns=["report_name", "content"])
# List text files in directory
file_names = os.listdir("reports_text")
text_file_names = [f[:-4] for f in file_names if f[-4:] == ".txt"]
# Append report name and report text for each report
for text_file_name in text_file_names:
    # Read text from report text file
    text_file = open(f"reports_text/{text_file_name}.txt", "r")
    content = text_file.read()
    content = (content
               .replace("\n", " ").replace("\r", " ").replace("\t", " ")
               .replace(",", "").replace(".", "").replace(":", "")
               .replace(";", "").replace("-", " ").replace("|", " ")
               .replace("(", "").replace(")", "").replace("?", "")
               .replace("!", "").replace("\'", "").replace('\"', ""))
    # print(content)
    text_file.close()    
    # Append report name and text to table
    report_row = pd.DataFrame([[text_file_name [:-5], content]], columns=["report_name", "content"])
    report_table = pd.concat([report_table, report_row],  axis=0, ignore_index=True)
# Save table as csv file
report_table.to_csv("reports.csv", index=False)

In [42]:
report_table.head()

,report_name,content
0,TRADERA,Climate Impact Report 2021 This is Traderas th...
1,SWEDBANK,1 SUSTAINABILITY POLICY Adopted by ...
2,HTC,ESG Repo rt 2022Environment Social Governance...
3,FISKARS,Sustainability Report 2022Contents Message fro...
4,KJELL & COMPANY,Kjell & Company ESG impacts throu...


# 4 Read Loughran-McDonald MasterD ictionary

In [3]:
master_dictionary = pd.read_csv("Loughran-McDonald_MasterDictionary_1993-2021.csv")

In [110]:
postive_words = master_dictionary[master_dictionary.Positive != 0]["Word"].to_list()
postive_words[:20]

['ABLE',
 'ABUNDANCE',
 'ABUNDANT',
 'ACCLAIMED',
 'ACCOMPLISH',
 'ACCOMPLISHED',
 'ACCOMPLISHES',
 'ACCOMPLISHING',
 'ACCOMPLISHMENT',
 'ACCOMPLISHMENTS',
 'ACHIEVE',
 'ACHIEVED',
 'ACHIEVEMENT',
 'ACHIEVEMENTS',
 'ACHIEVES',
 'ACHIEVING',
 'ADEQUATELY',
 'ADVANCEMENT',
 'ADVANCEMENTS',
 'ADVANCES']

In [111]:
negative_words = master_dictionary[master_dictionary.Negative != 0]["Word"].to_list()
negative_words[:20]

['ABANDON',
 'ABANDONED',
 'ABANDONING',
 'ABANDONMENT',
 'ABANDONMENTS',
 'ABANDONS',
 'ABDICATED',
 'ABDICATES',
 'ABDICATING',
 'ABDICATION',
 'ABDICATIONS',
 'ABERRANT',
 'ABERRATION',
 'ABERRATIONAL',
 'ABERRATIONS',
 'ABETTING',
 'ABNORMAL',
 'ABNORMALITIES',
 'ABNORMALITY',
 'ABNORMALLY']

# 5 Count Words in Reports

In [15]:
report_table = pd.read_csv("reports.csv")
report_table_count = report_table.copy()

In [120]:
def count_total_words(text):
    """Counts the total number of words in parameter string named text"""
    return sum([i.strip(string.punctuation).isalpha() for i in text.split()])

In [121]:
def count_words_in_text_from_list(text, word_list):
    """Counts the number of words in word_list in text"""
    # Initialize an empty list to store the word counts
    word_counts = {}
    text = text.upper()

    # Iterate through each word in the text
    for word in text.split():
        # Check if the word exists in the dictionary
        if word in word_counts:
            # Increment the count for the existing word
            word_counts[word] += 1
        else:
            # Add the word to the dictionary with a count of 1
            if word in word_list:
                word_counts[word] = 1
        
    return word_counts

In [122]:
report_table_count["word_count"] = report_table_count.content.map(count_total_words)
report_table_count["negative_word_count"] = report_table_count.content.map(lambda content: count_words_in_text_from_list(content, negative_words))
report_table_count["positive_word_count"] = report_table_count.content.map(lambda content: count_words_in_text_from_list(content, postive_words))
report_table_count["total_negative_word_count"] = report_table_count["negative_word_count"].map(lambda d: sum(d.values()))
report_table_count["total_positive_word_count"] = report_table_count["positive_word_count"].map(lambda d: sum(d.values()))

In [123]:
report_table_count = report_table_count.drop(columns="content")

In [124]:
report_table_count.to_csv("report_table_count.csv", index=False)

In [125]:
report_table_count.head()

,report_name,word_count,negative_word_count,positive_word_count,total_negative_word_count,total_positive_word_count
0,TRADERA,3888,"{'SLOW': 1, 'OVERESTIMATE': 1, 'UNDERESTIMATE'...","{'IMPROVED': 5, 'ABLE': 1, 'GREATER': 1, 'OPPO...",10,18
1,SWEDBANK,2402,"{'CORRUPTION': 2, 'CRUCIAL': 1, 'INFRINGING': ...","{'EMPOWERS': 1, 'BETTER': 5, 'POSITIVE': 1, 'O...",12,39
2,HTC,64382,"{'CHALLENGES': 12, 'CONFRONT': 1, 'CHALLENGE':...","{'ACHIEVEMENTS': 13, 'PROGRESS': 19, 'ASSURED'...",834,1149
3,FISKARS,29018,"{'AGAINST': 17, 'CATASTROPHE': 1, 'CONDEMNED':...","{'PROGRESS': 20, 'LEADING': 4, 'POSITIVE': 18,...",373,444
4,KJELL & COMPANY,1337,"{'CORRUPTION': 8, 'CONFLICT': 2, 'CRUCIAL': 2,...","{'LEADING': 1, 'SATISFACTION': 1, 'INTEGRITY':...",22,21


In [22]:
print(report_table_count.negative_word_count.map(lambda d: f"{max(d, key=d.get)}: {d[max(d, key=d.get)]}"))
print(report_table_count.positive_word_count.map(lambda d: f"{max(d, key=d.get)}: {d[max(d, key=d.get)]}"))
print(report_table_count[report_table_count.report_name == "IKEA"].negative_word_count.map(lambda d: f"{max(d, key=d.get)}: {d[max(d, key=d.get)]}"))
print(report_table_count[report_table_count.report_name == "IKEA"].positive_word_count.map(lambda d: f"{max(d, key=d.get)}: {d[max(d, key=d.get)]}"))

0          MISTAKE: 2
1       CORRUPTION: 2
2       HAZARDOUS: 39
3       OMISSIONS: 25
4       CORRUPTION: 8
            ...      
153    CORRUPTION: 22
154    CHALLENGES: 38
155     ACCIDENTS: 40
156      INCIDENT: 40
157       NEGATIVE: 9
Name: negative_word_count, Length: 158, dtype: object
0            IMPROVED: 5
1              BETTER: 5
2            IMPROVE: 67
3      OPPORTUNITIES: 31
4           INTEGRITY: 3
             ...        
153    OPPORTUNITIES: 22
154    OPPORTUNITIES: 79
155             GOOD: 42
156    OPPORTUNITIES: 57
157             ABLE: 39
Name: positive_word_count, Length: 158, dtype: object
118    CHALLENGES: 22
Name: negative_word_count, dtype: object
118    POSITIVE: 144
Name: positive_word_count, dtype: object


In [23]:
from collections import Counter

# Initialize counters for both negative and positive words
total_negative_words = Counter()
total_positive_words = Counter()

# Iterate through each row in the DataFrame and update the counters
for index, row in report_table_count.iterrows():
    total_negative_words.update(row['negative_word_count'])
    total_positive_words.update(row['positive_word_count'])

# Find the 50 most common negative and positive words
top_50_negative_words = total_negative_words.most_common(50)
top_50_positive_words = total_positive_words.most_common(50)

# Print the results
print("Top 50 Negative Words:")
for word, count in top_50_negative_words:
    print(f"{word}: {count}")

print("\nTop 50 Positive Words:")
for word, count in top_50_positive_words:
    print(f"{word}: {count}")

Top 50 Negative Words:
CORRUPTION: 1555
CHALLENGES: 1355
AGAINST: 1070
CRITICAL: 935
NEGATIVE: 831
INCIDENTS: 831
ACCIDENTS: 747
PREVENTION: 731
HAZARDOUS: 649
CHALLENGE: 640
LOSS: 607
CONCERNS: 597
FORCED: 561
FORCE: 498
VIOLATIONS: 483
CONFLICT: 478
INJURIES: 459
QUESTIONS: 420
CRISIS: 405
HARASSMENT: 402
INCIDENT: 400
BRIBERY: 381
SERIOUS: 375
ACCIDENT: 370
LOST: 362
INJURY: 315
CLAIMS: 305
STRESS: 304
CRUCIAL: 302
DAMAGE: 280
BREACHES: 279
CHALLENGING: 271
LOSSES: 271
PREVENTING: 271
LACK: 261
PROBLEMS: 261
COMPLAINTS: 260
VULNERABLE: 256
INVESTIGATION: 254
CONFLICTS: 245
DIFFICULT: 242
CONCERN: 240
HARMFUL: 240
DISCLOSE: 239
CLOSED: 231
ADVERSE: 228
HARM: 220
GRIEVANCE: 217
DISCLOSED: 214
ABSENCE: 210

Top 50 Positive Words:
PROGRESS: 2878
OPPORTUNITIES: 2801
IMPROVE: 2279
BETTER: 2269
EFFICIENCY: 2102
GOOD: 2012
LEADERSHIP: 1758
ACHIEVE: 1740
BEST: 1645
POSITIVE: 1607
COLLABORATION: 1587
INNOVATION: 1499
TRANSPARENCY: 1372
EFFICIENT: 1236
IMPROVEMENT: 1216
OPPORTUNITY: 1117
LEADI

# 6 Add Ranking

In [46]:
report_table_count = pd.read_csv("report_table_count.csv")
report_table_count["report_name"] = report_table_count.report_name.map(lambda r:  unicodedata.normalize("NFC", r))
pd.read_csv("report_table_count.csv")

,report_name,word_count,negative_word_count,positive_word_count,total_negative_word_count,total_positive_word_count
0,TRADERA,3888,"{'SLOW': 1, 'OVERESTIMATE': 1, 'UNDERESTIMATE'...","{'IMPROVED': 5, 'ABLE': 1, 'GREATER': 1, 'OPPO...",10,18
1,SWEDBANK,2402,"{'CORRUPTION': 2, 'CRUCIAL': 1, 'INFRINGING': ...","{'EMPOWERS': 1, 'BETTER': 5, 'POSITIVE': 1, 'O...",12,39
2,HTC,64382,"{'CHALLENGES': 12, 'CONFRONT': 1, 'CHALLENGE':...","{'ACHIEVEMENTS': 13, 'PROGRESS': 19, 'ASSURED'...",834,1149
3,FISKARS,29018,"{'AGAINST': 17, 'CATASTROPHE': 1, 'CONDEMNED':...","{'PROGRESS': 20, 'LEADING': 4, 'POSITIVE': 18,...",373,444
4,KJELL & COMPANY,1337,"{'CORRUPTION': 8, 'CONFLICT': 2, 'CRUCIAL': 2,...","{'LEADING': 1, 'SATISFACTION': 1, 'INTEGRITY':...",22,21
...,...,...,...,...,...,...
153,SANTANDER CONSUMER BANK,15397,"{'FORCE': 6, 'CHALLENGES': 1, 'QUESTION': 1, '...","{'PROGRESS': 11, 'OPPORTUNITIES': 22, 'GOOD': ...",224,215
154,SONY,112284,"{'CHALLENGES': 38, 'CRUCIAL': 4, 'DISASTER': 2...","{'CREATIVITY': 26, 'BEST': 26, 'INTEGRITY': 9,...",1011,1535
155,TULIP,35058,"{'QUESTIONS': 2, 'CHALLENGES': 9, 'SLOW': 1, '...","{'TRANSPARENCY': 9, 'PROGRESS': 11, 'ACHIEVEME...",364,410
156,UBER,40707,"{'FORCE': 7, 'CRITICAL': 34, 'INCIDENT': 40, '...","{'BEST': 32, 'PROFITABILITY': 1, 'ADVANTAGE': ...",471,720


In [47]:
ranking = pd.read_csv("sample.csv")
ranking.columns = ['ranking', 'Company', 'Koncern', 'Language', 'Type', 'Year of Report']
ranking = ranking[['ranking', 'Company']]
ranking["Company"] = ranking.Company.map(lambda company: company.upper())

In [48]:
report_table_count_ranking = report_table_count.merge(ranking, left_on="report_name", right_on="Company", how="inner")
report_table_count_ranking.drop(columns="Company", inplace=True)
report_table_count_ranking = report_table_count_ranking.sort_values("ranking").reset_index(drop=True)
report_table_count_ranking.to_csv("report_table_count_ranking.csv", index=False)

# 7 Add Industries

In [49]:
report_table_count = pd.read_csv("report_table_count.csv")
report_table_count["report_name"] = report_table_count.report_name.map(lambda r:  unicodedata.normalize("NFC", r))
report_table_industries = pd.read_csv("report_table_industries.csv")
report_table_industries["Company"] = report_table_industries["Company"].map(lambda name: unicodedata.normalize("NFC", str(name)).upper().strip())

In [50]:
# Set the columns names to be consistent for the merge
report_table_industries.columns = ['Company', 'industry']
# Merge the two tables on the 'Company' column, using an inner join
merged_table = report_table_count.merge(report_table_industries, left_on="report_name", right_on="Company", how="inner")
# Drop the redundant 'Company' column from the merged table
merged_table.drop(columns="Company", inplace=True)
# Sort, reset index, and save the new table to CSV
merged_table = merged_table.sort_values(by=["industry", "report_name"]).reset_index(drop=True)
merged_table.to_csv("report_table_count_industries.csv", index=False)

In [51]:
# Load the industries table
report_table_industries = pd.read_csv("report_table_industries.csv")
# Load the count table
report_table_count_industries = pd.read_csv("report_table_count_industries.csv")

def find_missing_companies(industries_df, count_df, industries_column, count_column):
    # Convert both columns to upper case to ensure matching is case-insensitive
    industries_df[industries_column] = industries_df[industries_column].str.upper()
    count_df[count_column] = count_df[count_column].str.upper()
    # Find companies in industries_df that are not in count_df
    missing_companies = set(industries_df[industries_column]) - set(count_df[count_column])
    
    return missing_companies


missing_companies = find_missing_companies(report_table_industries, report_table_count_industries, 'Company', 'report_name')
# Print or return the missing companies
print(missing_companies)

set()


In [52]:
report_table_count_ranking = pd.read_csv("report_table_count_ranking.csv")
report_table_count_ranking["report_name"] = report_table_count_ranking.report_name.map(lambda r:  unicodedata.normalize("NFC", r))
report_table_all_industries = pd.read_csv("report_table_all_industries.csv")
report_table_all_industries["Company"] = report_table_all_industries["Company"].map(lambda name: unicodedata.normalize("NFC", str(name)).upper().strip())

In [53]:
# Set the columns names to be consistent for the merge
report_table_all_industries.columns = ['Company', 'industry']
# Merge the two tables on the 'Company' column, using an inner join
merged_table = report_table_count_ranking.merge(report_table_all_industries, left_on="report_name", right_on="Company", how="inner")
# Drop the redundant 'Company' column from the merged table
merged_table.drop(columns="Company", inplace=True)
# Sort, reset index, and save the new table to CSV
merged_table = merged_table.sort_values(by=["industry", "report_name"]).reset_index(drop=True)
merged_table.to_csv("report_table_count_all_industries.csv", index=False)

# Add Financial Metrics

In [37]:
report_table_count_all_industries = pd.read_csv("report_table_count_all_industries.csv")
report_table_count_all_industries["report_name"] = report_table_count_all_industries.report_name.map(lambda r:  unicodedata.normalize("NFC", r))
report_table_financial_metrics = pd.read_csv("report_table_financial_metrics.csv")
report_table_financial_metrics["report_name"] = report_table_financial_metrics["report_name"].map(lambda name: unicodedata.normalize("NFC", str(name)).upper().strip())

In [38]:
# Set the columns names to be consistent for the merge
report_table_financial_metrics.columns = ['report_name', 'total_assets', 'roa', 'publicly_traded']
# Merge the two tables on the 'report_name' column, using an inner join
merged_table = report_table_count_all_industries.merge(report_table_financial_metrics, left_on="report_name", right_on="report_name", how="inner")
# Sort, reset index, and save the new table to CSV
merged_table = merged_table.sort_values(by=["industry", "report_name"]).reset_index(drop=True)
merged_table.to_csv("report_table_count_all_control_variables.csv", index=False)

In [2]:
allcv = pd.read_csv('report_table_count_all_control_variables.csv')

In [6]:
allcv['roa'] = allcv['roa'].astype(str).str.replace(',', '.').astype(float)

In [7]:
allcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   report_name                156 non-null    object 
 1   word_count                 156 non-null    int64  
 2   negative_word_count        156 non-null    object 
 3   positive_word_count        156 non-null    object 
 4   total_negative_word_count  156 non-null    int64  
 5   total_positive_word_count  156 non-null    int64  
 6   ranking                    156 non-null    int64  
 7   industry                   156 non-null    object 
 8   total_assets               156 non-null    int64  
 9   roa                        156 non-null    float64
 10  publicly_traded            156 non-null    int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 13.5+ KB


In [8]:
allcv.head()

,report_name,word_count,negative_word_count,positive_word_count,total_negative_word_count,total_positive_word_count,ranking,industry,total_assets,roa,publicly_traded
0,BRA,17639,"{'STRINGENT': 4, 'DIFFICULT': 1, 'HARSH': 1, '...","{'STRENGTHEN': 7, 'HIGHEST': 17, 'OPPORTUNITIE...",156,340,356,Airlines,1512,-0.07,0
1,BRITISH AIRWAYS,5882,"{'CHALLENGE': 2, 'AGAINST': 1, 'CRITICAL': 1, ...","{'INCREDIBLE': 1, 'GREATEST': 2, 'ACHIEVEMENTS...",21,114,388,Airlines,42842,0.12,0
2,LUFTHANSA,12702,"{'CHALLENGING': 2, 'DISADVANTAGED': 1, 'FORCE'...","{'SUCCESSFUL': 2, 'PLEASED': 1, 'LEADING': 5, ...",81,180,349,Airlines,507770,0.04,1
3,RYANAIR,16813,"{'DEVASTATED': 1, 'DISRUPTED': 1, 'SUFFERED': ...","{'SATISFACTION': 7, 'ADVANTAGE': 5, 'GAINS': 1...",148,203,416,Airlines,173111,0.13,1
4,BMW,7038,"{'AGAINST': 3, 'DEVIATIONS': 1, 'FORCE': 1, 'C...","{'TRANSPARENCY': 26, 'EFFECTIVE': 22, 'SUCCESS...",30,139,227,Automotives,2858751,0.04,1


# 9 Calculate Statistics

In [14]:
# Regression with all control variables

In [30]:
# Load the CSV file
df = pd.read_csv('report_table_count_all_control_variables.csv')

In [31]:
# Calculate 'tone'
df['tone'] = (df['total_positive_word_count'] - df['total_negative_word_count']) / (df['total_positive_word_count'] + df['total_negative_word_count'])

In [32]:
# Convert 'industry' to dummy variables
industry_dummies = pd.get_dummies(df['industry'], prefix='industry', drop_first=True)

# Convert boolean dummy variables to int
for column in industry_dummies.columns:
    industry_dummies[column] = industry_dummies[column].astype(int)

df = pd.concat([df, industry_dummies], axis=1)

In [33]:
# Convert 'roa' to string, replace commas with periods, then convert to float
df['roa'] = df['roa'].astype(str).str.replace(',', '.').astype(float)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 41 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   report_name                             156 non-null    object 
 1   word_count                              156 non-null    int64  
 2   negative_word_count                     156 non-null    object 
 3   positive_word_count                     156 non-null    object 
 4   total_negative_word_count               156 non-null    int64  
 5   total_positive_word_count               156 non-null    int64  
 6   ranking                                 156 non-null    int64  
 7   industry                                156 non-null    object 
 8   total_assets                            156 non-null    int64  
 9   roa                                     156 non-null    float64
 10  publicly_traded                         156 non-null    int64 

In [36]:
# Define the dependent variable
y = df['tone']

# Define the independent variables
X = df[['ranking', 'total_assets', 'roa', 'publicly_traded'] + list(industry_dummies.columns)]

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Initialize and fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   tone   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     1.591
Date:                Fri, 22 Mar 2024   Prob (F-statistic):             0.0366
Time:                        20:15:22   Log-Likelihood:                 48.617
No. Observations:                 156   AIC:                            -29.23
Df Residuals:                     122   BIC:                             74.46
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [15]:
# Other regressions

In [38]:
# Load the CSV file
all_control = pd.read_csv('report_table_count_all_control_variables.csv')

# Calculate 'tone'
all_control['tone'] = (all_control['total_positive_word_count'] - all_control['total_negative_word_count']) / (all_control['total_positive_word_count'] + all_control['total_negative_word_count'])

# Convert 'industry' to dummy variables
industry_dummies = pd.get_dummies(all_control['industry'], prefix='industry', drop_first=True)

# Convert boolean dummy variables to int
for column in industry_dummies.columns:
    industry_dummies[column] = industry_dummies[column].astype(int)

all_control = pd.concat([all_control, industry_dummies], axis=1)

# Convert 'roa' to string, replace commas with periods, then convert to float
all_control['roa'] = all_control['roa'].astype(str).str.replace(',', '.').astype(float)

In [39]:
# Define the dependent variable
y = all_control['tone']

# Define the independent variables
X = all_control[['total_assets', 'roa', 'publicly_traded'] + list(industry_dummies.columns)]

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Initialize and fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   tone   R-squared:                       0.296
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     1.612
Date:                Fri, 22 Mar 2024   Prob (F-statistic):             0.0338
Time:                        21:21:50   Log-Likelihood:                 48.025
No. Observations:                 156   AIC:                            -30.05
Df Residuals:                     123   BIC:                             70.60
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [54]:
df = pd.read_csv("report_table_count_ranking.csv")
df['total_positive_word_count'] = pd.to_numeric(df['total_positive_word_count'], errors='coerce')
df['total_negative_word_count'] = pd.to_numeric(df['total_negative_word_count'], errors='coerce')

In [55]:
# Calculate 'tone'
df['tone'] = (df['total_positive_word_count'] - df['total_negative_word_count']) / (df['total_positive_word_count'] + df['total_negative_word_count'])

In [56]:
# Assign rankings based on row order
df['rank'] = df.index + 1
# Rank the 'tone' column to get 'tone_rank'
df['tone_rank'] = df['tone'].rank(ascending=False, method='min')
# Select the 'report_name', 'rank', 'tone', and 'tone_rank' columns for the new CSV
df_new = df[['report_name', 'rank', 'tone', 'tone_rank']]
# Save to a new CSV file named "report_table_tone_rank.csv"
df_new.to_csv("report_table_tone_rank.csv", index=False)

In [57]:
# Calculate Spearman's correlation
spearman_corr, p_value = spearmanr(df['rank'], df['tone_rank'])
# Output the results
print(f"Spearman's correlation coefficient: {spearman_corr}")
print(f"p-value: {p_value}")

Spearman's correlation coefficient: 0.04155117911401642
p-value: 0.6042022231512534


In [58]:
df = pd.read_csv("report_table_count_industries.csv")
df['total_positive_word_count'] = pd.to_numeric(df['total_positive_word_count'], errors='coerce')
df['total_negative_word_count'] = pd.to_numeric(df['total_negative_word_count'], errors='coerce')

In [59]:
# Calculate 'tone'
df['tone'] = (df['total_positive_word_count'] - df['total_negative_word_count']) / (df['total_positive_word_count'] + df['total_negative_word_count'])

In [60]:
# Group the data by industry and collect tones into lists
tones_by_industry = df.groupby('industry')['tone'].apply(list)
# Prepare the arguments for the Kruskal-Wallis test
args = [group for group in tones_by_industry]
# Perform the Kruskal-Wallis test
stat, p_value = kruskal(*args)
# Output the results
print(f"Kruskal-Wallis test statistic: {stat}")
print(f"p-value: {p_value}")

Kruskal-Wallis test statistic: 27.615509407711862
p-value: 0.09111271506349974


In [2]:
# Load the CSV file
df = pd.read_csv('report_table_count_all_industries.csv')

# Convert 'ranking' to numeric, handling any non-numeric values as NaN
df['ranking'] = pd.to_numeric(df['ranking'], errors='coerce')

# Ensure 'total_positive_word_count' and 'total_negative_word_count' are numeric
df['total_positive_word_count'] = pd.to_numeric(df['total_positive_word_count'], errors='coerce')
df['total_negative_word_count'] = pd.to_numeric(df['total_negative_word_count'], errors='coerce')

# Calculate 'tone'
df['tone'] = (df['total_positive_word_count'] - df['total_negative_word_count']) / (df['total_positive_word_count'] + df['total_negative_word_count'])

In [5]:
# Generate dummy variables for the 'industry' column, excluding the first to avoid multicollinearity
industry_dummies = pd.get_dummies(df['industry'], drop_first=True)

# Prepare the independent variables (including industry dummies) and the dependent variable
X = pd.concat([df[['ranking']], industry_dummies], axis=1)
y = df['tone']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Convert boolean dummy variables to int
for column in industry_dummies.columns:
    industry_dummies[column] = industry_dummies[column].astype(int)

# If you've already concatenated the dummy variables to X, convert them directly in X
for column in X.columns[2:]:  # Assuming the first two columns are 'const' and 'ranking'
    X[column] = X[column].astype(int)

# Now, X and y should be fully compatible with statsmodels
# Fit the OLS regression model again
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   tone   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     1.545
Date:                Fri, 01 Mar 2024   Prob (F-statistic):             0.0511
Time:                        02:36:42   Log-Likelihood:                 45.051
No. Observations:                 158   AIC:                            -28.10
Df Residuals:                     127   BIC:                             66.84
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

In [3]:
# Generate dummy variables for the 'industry' column, excluding the first to avoid multicollinearity
industry_dummies = pd.get_dummies(df['industry'], drop_first=True)

# Prepare the independent variables (including industry dummies) and the dependent variable
X = pd.concat([df[['tone']], industry_dummies], axis=1)
y = df['ranking']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Convert boolean dummy variables to int
for column in industry_dummies.columns:
    industry_dummies[column] = industry_dummies[column].astype(int)

# If you've already concatenated the dummy variables to X, convert them directly in X
for column in X.columns[2:]:  # Assuming the first two columns are 'const' and 'ranking'
    X[column] = X[column].astype(int)

# Now, X and y should be fully compatible with statsmodels
# Fit the OLS regression model again
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                ranking   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     1.455
Date:                Sun, 03 Mar 2024   Prob (F-statistic):             0.0792
Time:                        00:46:22   Log-Likelihood:                -955.76
No. Observations:                 158   AIC:                             1974.
Df Residuals:                     127   BIC:                             2068.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

In [3]:
# Load the CSV file
df = pd.read_csv('report_table_count_all_industries.csv')

# Convert 'ranking' to numeric, handling any non-numeric values as NaN
df['ranking'] = pd.to_numeric(df['ranking'], errors='coerce')

# Ensure 'total_positive_word_count' and 'total_negative_word_count' are numeric
df['total_positive_word_count'] = pd.to_numeric(df['total_positive_word_count'], errors='coerce')
df['total_negative_word_count'] = pd.to_numeric(df['total_negative_word_count'], errors='coerce')
df['total_negative_word_count'] = pd.to_numeric(df['word_count'], errors='coerce')

# Calculate 'tone'
df['tone_%'] = (df['total_positive_word_count'] / df['word_count'])/ - (df['total_negative_word_count'] / df['word_count'])

In [6]:
# Generate dummy variables for the 'industry' column, excluding the first to avoid multicollinearity
industry_dummies = pd.get_dummies(df['industry'], drop_first=True)

# Prepare the independent variables (including industry dummies) and the dependent variable
X = pd.concat([df[['ranking', 'word_count']], industry_dummies], axis=1)
y = df['tone']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Convert boolean dummy variables to int
for column in industry_dummies.columns:
    industry_dummies[column] = industry_dummies[column].astype(int)

# If you've already concatenated the dummy variables to X, convert them directly in X
for column in X.columns[2:]:  # Assuming the first two columns are 'const' and 'ranking'
    X[column] = X[column].astype(int)

# Now, X and y should be fully compatible with statsmodels
# Fit the OLS regression model again
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   tone   R-squared:                       0.277
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     1.557
Date:                Fri, 01 Mar 2024   Prob (F-statistic):             0.0464
Time:                        02:36:49   Log-Likelihood:                 46.099
No. Observations:                 158   AIC:                            -28.20
Df Residuals:                     126   BIC:                             69.81
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

In [7]:
# Assuming df is your DataFrame loaded from the CSV file
# Assuming 'ranking' and 'tone' have already been calculated and are present in df

# Convert 'ranking' to numeric, handling any non-numeric values as NaN
df['ranking'] = pd.to_numeric(df['ranking'], errors='coerce')

# Prepare the independent variable ('ranking') and the dependent variable ('tone')
X = df[['ranking']]
y = df['tone']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the OLS regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   tone   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2775
Date:                Fri, 01 Mar 2024   Prob (F-statistic):              0.599
Time:                        02:36:53   Log-Likelihood:                 20.616
No. Observations:                 158   AIC:                            -37.23
Df Residuals:                     156   BIC:                            -31.11
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2359      0.035      6.680      0.0

In [ ]:
# Comment Jonas